In [ ]:
# Install necessary libraries and related packages
!pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu118
!pip install transformers==4.46.3 tokenizers==0.20.3 einops addict easydict
!pip install flash-attn==2.7.3 --no-build-isolation
!pip install safetensors

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch
import os

# Define the model name.
model_name = 'deepseek-ai/DeepSeek-OCR'

# Load the tokenizer.
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Load the pre-trained model with specific configurations
model = AutoModel.from_pretrained(
    model_name,
    trust_remote_code=True,
    use_safetensors=True,
    attn_implementation="eager",
    torch_dtype=torch.bfloat16,
)

# Set the model to evaluation mode and move it to the CUDA device.
model = model.eval().cuda()

In [ ]:
# Define the prompt for the OCR task.
prompt = "<image>\n<|grounding|>Convert the document to Markdown"

image_file = '/Resume.png'

output_path = '/content/'

# Perform inference using the model:
res = model.infer(tokenizer, prompt=prompt, image_file=image_file, output_path = output_path, base_size = 1024, image_size = 640, crop_mode=True, save_results = True, test_compress = True)

In [ ]:
with open("result.mmd", "r", encoding="utf-8") as f:
    # Read the entire content of the file into the 'ocr_text' variable.
    ocr_text = f.read()

# Print the first 50 characters of the extracted text to preview the content.
print(ocr_text[:50])

In [ ]:
pip install agno

In [ ]:
# Import necessary libraries: os for environment variables, Agent from agno, and Gemini model wrapper.
import os
from agno.agent import Agent
from agno.models.google import Gemini
from google.colab import userdata

api_key = userdata.get('GOOGLE_API_KEY')

agent = Agent(
    name="ResumeExtractor",
    model=Gemini(id="gemini-2.0-flash-exp", api_key=api_key),
    description="You are an expert resume parser that extracts structured information from resume text.",
    instructions="Extract specific information from resume text and return it in a structured JSON format.",
    markdown=False,
)

try:

    extraction_prompt = f"""
Extract the following information from this resume and return it as a valid JSON object:

REQUIRED FIELDS:
1. name
2. skills
3. experience

INSTRUCTIONS:
- Extract all available information accurately
- If a field is not found, use empty string "" or empty array []
- Ensure proper JSON formatting with double quotes
- Do not include any markdown formatting or code blocks
- Return only the JSON object, nothing else

RESUME TEXT:
{ocr_text}

Extract the information now:
"""
    #Executing the agent
    response = agent.run(extraction_prompt)

    print(f"\nAgent: {response.content}\n")

except Exception as e:
    print(f"An error occurred: {e}")
    print("Please try again.")